# Bloque 1: Importaciones y Configuración

- Propósito: Cargar las librerías necesarias (numpy, librosa, joblib, os).
También definimos las variables clave:
1) MODELO_PATH: Dónde está guardado nuestro modelo entrenado.
2) AUDIO_PARA_CLASIFICAR: El nombre del archivo de audio que queremos probar.
3) CLASES_MAP: Un "diccionario" para traducir la predicción numérica (0, 1, 2) a texto.
4) N_MFCC: El número de características que usamos (debe ser el mismo del entrenamiento).

In [62]:
# Importaciones
import numpy as np
import librosa
import joblib
import os 
MODELO_PATH = "clasificador_vehiculos.joblib"
AUDIO_PARA_CLASIFICAR = "audio_random.wav" 

# Mapeo de las etiquetas numéricas a texto
CLASES_MAP = {0: "pequeno", 1: "mediano", 2: "grande"} 
N_MFCC = 40 


# Bloque 2: Función de Extracción de Características

- Propósito: Definir la función que convierte un archivo de audio en números.
Esta función DEBE ser una copia exacta de la que se usó en el notebook 1.
Carga el audio, calcula los 40 MFCCs, y luego calcula la media de esos MFCCs
para obtener un solo vector de características.

In [63]:
# Bloque 2: Función de Extracción de Características (CORREGIDA PARA 80 FEATURES)
def extract_features(file_path, n_mfcc=40):
    """
    Carga un archivo de audio y extrae Media + Desviación Estándar.
    Total: 80 características (40 media + 40 std).
    """
    try:
        y, sr = librosa.load(file_path, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        
        # 1. Calcular la Media (40 datos)
        mfccs_mean = np.mean(mfccs.T, axis=0)
        
        # 2. Calcular la Desviación Estándar (40 datos)
        mfccs_std = np.std(mfccs.T, axis=0)
        
        # 3. Unirlos para tener los 80 datos que espera el modelo
        features_vector = np.hstack([mfccs_mean, mfccs_std])
        
    except Exception as e:
        print(f"Error cargando {file_path}: {e}")
        return None
        
    return features_vector

# Bloque 3: Carga del Modelo

- Propósito: Cargar en memoria el modelo .joblib que creamos en el notebook 1.
'joblib.load()' lee el archivo y lo convierte de nuevo en un objeto de Python
(nuestro clasificador 'model') que está listo para hacer predicciones.

In [64]:
# Bloque 3: Carga del Modelo
print(f"Cargando modelo desde: {MODELO_PATH}")
try:
    model = joblib.load(MODELO_PATH)
    print("¡Modelo cargado exitosamente!")
except FileNotFoundError:
    print(f"ERROR: No se encontró el archivo '{MODELO_PATH}'.")
    print("Asegúrate de ejecutar el Notebook '01_Entrenamiento_Modelo.ipynb' primero.")
    raise

Cargando modelo desde: clasificador_vehiculos.joblib
¡Modelo cargado exitosamente!


# Bloque 4: Clasificación del Audio

- Propósito: Usar el modelo

1) Verifica si el archivo de audio existe.
2) Llama a la función 'extract_features' (del Bloque 2) para convertir el audio en un vector.
3) Usa 'model.predict()' para obtener la predicción (que será 0, 1 o 2).
4) Usa 'CLASES_MAP' (del Bloque 1) para traducir el número a "pequeno", "mediano" o "grande".
5) Muestra el resultado final al usuario.

In [65]:
# Bloque 4: Clasificación del Audio
print(f"\nAnalizando audio: {AUDIO_PARA_CLASIFICAR}...")

# Verificar que el audio exista
if not os.path.exists(AUDIO_PARA_CLASIFICAR):
    print(f"ERROR: No se encontró el archivo de audio '{AUDIO_PARA_CLASIFICAR}'.")
else:
    # Extraer características del audio (usando la misma función)
    features = extract_features(AUDIO_PARA_CLASIFICAR, n_mfcc=N_MFCC)
    
    if features is not None:
        # Preparar el vector para el modelo
        features_2d = features.reshape(1, -1)
        
        # Predecir
        prediccion_num = model.predict(features_2d)
        
        # Obtener el resultado en texto
        clase_predicha = CLASES_MAP[prediccion_num[0]]
        
        # Mostrar el resultado
        print("\n===============================")
        print("     ✅ RESULTADO ✅")
        print("===============================")
        print(f"El audio '{AUDIO_PARA_CLASIFICAR}' ha sido clasificado como:")
        print(f" -> **{clase_predicha.upper()}**")
        print("===============================")
    else:
        print("No se pudieron extraer características del audio.")


Analizando audio: audio_random.wav...

     ✅ RESULTADO ✅
El audio 'audio_random.wav' ha sido clasificado como:
 -> **GRANDE**


EXPLIACION

In [66]:
import numpy as np
import librosa
import joblib
import os 

MODELO_PATH = "clasificador_vehiculos.joblib"
AUDIO_PARA_CLASIFICAR = "audio_random.wav" 

CLASES_MAP = {0: "pequeno", 1: "mediano", 2: "grande"} 
N_MFCC = 40 


SEGMENT_DUR_SEC = 0.5  # Duración de la ventana de análisis (ej: 1 segundo)
OVERLAP_RATIO = 0.8    # Solapamiento entre ventanas (50%)
MIN_EVENT_SEGMENTS = 5 # Mínimo de segmentos consecutivos para contar como evento único

EXPLICACION

In [67]:
# Bloque 2: Función de extracción de características (VERSIÓN MEJORADA 80 FEATURES)
# Bloque 2 del NOTEBOOK 2 (Aplicación/Conteo)
def extract_features_segment(y_segment, sr, n_mfcc=40):
    """Recibe un array de audio y calcula 80 características (Media + Std)."""
    try:
        # 1. Calcular MFCCs (Directamente sobre el segmento)
        mfccs = librosa.feature.mfcc(y=y_segment, sr=sr, n_mfcc=n_mfcc)
        
        # 2. Calcular Media y Desviación (IGUAL QUE EN EL NOTEBOOK 1)
        mfccs_mean = np.mean(mfccs.T, axis=0)
        mfccs_std = np.std(mfccs.T, axis=0)
        
        # 3. Unir ambos
        features_vector = np.hstack([mfccs_mean, mfccs_std]) 
        
    except Exception as e:
        print(f"Error procesando segmento: {e}")
        return None
    return features_vector

EXPLICACION

In [68]:
import numpy as np
import librosa
import joblib
import os
from scipy.stats import mode

# --- 1. FUNCIÓN DE EXTRACCIÓN INTERNA (CORREGIDA A 80 FEATURES) ---
def extract_features_segment(y_segment, sr, n_mfcc=40):
    try:
        mfccs = librosa.feature.mfcc(y=y_segment, sr=sr, n_mfcc=n_mfcc)
        
        # Calcular MEDIA (40 features)
        mfccs_mean = np.mean(mfccs.T, axis=0)
        
        # Calcular DESVIACIÓN ESTÁNDAR (40 features)
        mfccs_std = np.std(mfccs.T, axis=0)
        
        # Juntar ambos para obtener 80 features
        features_vector = np.hstack([mfccs_mean, mfccs_std])
        
        return features_vector
    except Exception as e:
        print(f"Error procesando segmento: {e}")
        return None

# --- 2. CONFIGURACIÓN ---
print(f"\nAnalizando audio largo: {AUDIO_PARA_CLASIFICAR}...")

if not os.path.exists(AUDIO_PARA_CLASIFICAR):
    print(f"ERROR: No se encontró el archivo '{AUDIO_PARA_CLASIFICAR}'.")
else:
    # Cargar modelo
    try:
        model = joblib.load(MODELO_PATH)
    except Exception as e:
        print(f"Error cargando modelo: {e}")
        raise 

    # Cargar audio
    y, sr = librosa.load(AUDIO_PARA_CLASIFICAR, res_type='kaiser_fast')
    
    # Configurar Ventana
    segment_len = int(SEGMENT_DUR_SEC * sr)
    hop_len = int(segment_len * (1 - OVERLAP_RATIO))

    features_list = []
    
    # --- 3. EXTRACCIÓN POR SEGMENTOS ---
    for i in range(0, len(y) - segment_len, hop_len):
        y_segment = y[i:i + segment_len]
        # Llamamos a la función de 80 features
        features = extract_features_segment(y_segment, sr, n_mfcc=N_MFCC)
        if features is not None:
            features_list.append(features)

    if not features_list:
        print("Error: No se extrajeron segmentos.")
    else:
        # --- 4. PREDICCIÓN ---
        X_segments = np.array(features_list)
        
        # AHORA SÍ: X_segments tiene shape (N, 80) y el modelo estará feliz
        try:
            predicciones_raw = model.predict(X_segments)
        except ValueError as ve:
            print(f"\n❌ ERROR DE DIMENSIONES: {ve}")
            raise

        # --- 5. SUAVIZADO Y CONTEO ---
        predicciones_suavizadas = []
        ventana_suave = 5 
        
        for i in range(len(predicciones_raw)):
            inicio = max(0, i - ventana_suave // 2)
            fin = min(len(predicciones_raw), i + ventana_suave // 2 + 1)
            fragmento = predicciones_raw[inicio:fin]
            
            moda_res = mode(fragmento, keepdims=True)
            predicciones_suavizadas.append(moda_res.mode[0])
            
        predicciones = predicciones_suavizadas

        # Conteo Lógico
        conteo_final = {clase: 0 for clase in CLASES_MAP.values()}
        event_class = -1
        consecutive_count = 0
        
        for pred in predicciones:
            if pred == event_class:
                consecutive_count += 1
            else:
                if event_class != -1 and consecutive_count >= MIN_EVENT_SEGMENTS:
                    clase_texto = CLASES_MAP[event_class]
                    conteo_final[clase_texto] += 1
                
                event_class = pred
                consecutive_count = 1
        
        # Último evento
        if event_class != -1 and consecutive_count >= MIN_EVENT_SEGMENTS:
            clase_texto = CLASES_MAP[event_class]
            conteo_final[clase_texto] += 1
            
        # --- 6. RESULTADOS ---
        print("\n===============================")
        print("      RESULTADO FINAL DE CONTEO")
        print("===============================")
        print(f"Audio: {AUDIO_PARA_CLASIFICAR} ({len(y)/sr:.2f} s)")
        print(f"Segmentos analizados: {len(features_list)}")
        print("---")
        for clase, count in conteo_final.items():
            print(f"Vehículos tipo **{clase.upper()}**: {count}")
        print("===============================")


Analizando audio largo: audio_random.wav...

      RESULTADO FINAL DE CONTEO
Audio: audio_random.wav (13.47 s)
Segmentos analizados: 130
---
Vehículos tipo **PEQUENO**: 1
Vehículos tipo **MEDIANO**: 7
Vehículos tipo **GRANDE**: 1


EXPLICACION